In [1]:
import pandas as pd
import numpy as np
import PyPDF2
import os
import re

In [11]:
import docx

In [2]:
from nltk.corpus import wordnet

# Get all the PDF and word docx filenames

In [3]:
pdf_files = []

In [5]:
word_files = []

In [6]:
for filename in os.listdir('Cvs/'):
    if filename.endswith('.pdf'):
         pdf_files.append('Cvs/'+filename)
    else:
        word_files.append('Cvs/'+filename)

# Number of pdf files collected

In [7]:
len(pdf_files)

2644

In [8]:
len(word_files)

15

# Process PDF

In [6]:
from PyPDF2 import utils
def extract_text_from_pdf():
    res = {}
    for pdf in pdf_files:
        pdfFileObj = open(pdf, 'rb')
        try:
            pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        except utils.PdfReadError:
            print(pdf, " not processed")
            # pdfReader = PyPDF2.PdfFileReader(decompress_pdf(input_file))
        if not pdfReader.isEncrypted:
            num_pages = pdfReader.numPages
            pages = []
            for page_nb in range(num_pages):
                pages.append(pdfReader.getPage(page_nb).extractText())
            res[pdf] = ' '.join(pages)
    return res

In [7]:
pdf_contents = extract_text_from_pdf()

Cvs/Cerniute_Monika_20190320084240.pdf  not processed
Cvs/Hu_Xuan_20190320090349.pdf  not processed


Cvs/Sharp_Kristen_20190320083417.pdf  not processed


Cvs/Wong_Terence_20190320085947.pdf  not processed


In [91]:
with open('stopwords.txt') as stopfile:
    stopwords = stopfile.read()
    stop = stopwords.split()

In [92]:
def clean_word(x):
    res = []
    replace_list = [
        (",", ""), ("-", " "), ("  ", " "),
        ("(", " "), (")", " "),
        (" - ", " "), (" / ", " "), ("/", " "),
        ("¥", ""), ("% ", ""),
        (",", " "), ("?", " "), ("!", " "),
        ("   ", "")
    ] 
    rx = re.compile(r"<.*?>", flags=re.DOTALL)

    for content in x:
        for pattern, replacement in replace_list:
            content = content.replace(pattern, replacement)

        content = rx.sub("", content)
        # content = ''.join([i for i in content if not i.isdigit()])
        if len(content)>1:
            res.append(content)
    return res

In [93]:
def remove_stopwords(sentences):
    res = []
    for sentence in sentences:
        a = ''
        x = sentence.split(' ')
        for word in x:
            if word not in stop:
                a += word + ' '
        res.append(a)
    return res

In [94]:
def separate_uppercase(sentences):
    res = []
    for sentence in sentences:
        words = sentence.split(' ')
        a = 0
        if words[a].isupper():
            a += 1
            while words[a].isupper():
                a += 1
            res.append(' '.join(words[:a]))
            res.append(' '.join(words[a:]))
        else:
            res.append(' '.join(words))
    return res

In [95]:
def clean_pdf():
    res = {}
    for name, content in pdf_contents.items():
        x = content.split('\n')
        x = [i for i in x if len(i)>1]
        x = [i.rstrip() for i in x if bool(re.search('[a-zA-Z]+', i))]
        n = len(x)
        for i, text in enumerate(x):
            if i > 0:
                if len(text) > 1:
                    if text[0].isupper() or text[1].isupper():
                        x[i] = '. ' + text
            if i < n - 1:
                next_text = x[i+1]
                first_word = next_text.split(' ')[0]
                if first_word.isdigit() or len(first_word)>1:
                    x[i+1] = ' ' + next_text
        pdf = ''.join(x)
        sentences = pdf.split('.')
        sentences = [i.strip() for i in sentences]
        x = []
        for sentence in sentences:
            words = sentence.split(' ')
            a = ''
            for word in words:
                if len(word)>1:
                    a += word + ' '
            x.append(a)
        sentences = remove_stopwords(x)
        sentences = separate_uppercase(sentences)
        sentences = clean_word(sentences)
        res[name] = sentences
    return res   

In [13]:
def get_section_titles(x):
    res = []
    trash = []
    for name, content in x.items():
        for sentence in content:
            if sentence.isupper():
                words = sentence.split(" ")
                a = []
                for word in words:
                    if len(word) > 2:
                        if wordnet.synsets(word.lower()):
                            a.append(word.lower())
                if len(a) > 1:
                    content = ' '.join(a)
                    content = ''.join([i for i in content if not i.isdigit()])
                    if len(content) > 5 :
                        res.append(content)
    final = list(set(res))
    final.append('skills')
    final.append('eduction')
    final.append('profile')
    return final

In [14]:
cleaned_pdf = clean_pdf()

In [20]:
titles = get_section_titles(cleaned_pdf)

In [21]:
titles

['language computer skills',
 'experience related experiences',
 'management project',
 'experience work',
 'graduate engineer',
 'john ambulance',
 'covenant university',
 'university football committee',
 'contact information',
 'jakarta development corporation',
 'curricular skills',
 'engineering consultant',
 'career summary',
 'profile summary',
 'south eastern kenya university',
 'mark net',
 'university expected',
 'university west london',
 'additional qualifications',
 'vice captain',
 'hour race',
 'clean driving licence',
 'management june',
 'scuba diving',
 'foreign computer languages',
 'other activities responsibilities',
 'group london',
 'project portfolio',
 'long investment company china',
 'notable modules',
 'september may',
 'career events',
 'volunteering experience',
 'day visit',
 'business experience',
 'hiking road cycling trail sewing kite',
 'daniel diaz',
 'scholarships awards',
 'professor assistant checker',
 'additional experience',
 'business school i

In [22]:
len(titles)

811

In [23]:
cleaned_pdf[pdf_files[340]]

['HENRY',
 'EWBANK',
 'Address: 56 Crossgate Durham DH1 4PR United Kingdom ',
 'Mobile: +44  0 7754 183 340 Email: harryewbank@btinternet ',
 'Final ear Natural Sciences student Durham University exposure consultative selling working alongside former consultants summer internship Potentialife ',
 'Excited prospect combining quantitative skills scientific background witha passion collaboration cultivated leadership experiences graduate po sition Alpha ',
 'Education ',
 'BSc Natural Science Durham University ',
 'September 2016 ',
 'Present Expected Grade ',
 'Relevant Modules: ',
 'Mathematical Experimental Tools Required In Chemistry ',
 'Involves advanced calculus algebra statistical analysis among topics ',
 ' 1 st year  ',
 'Entrepreneurship ',
 'Understanding theories models frameworks required high performance upon inception new business ',
 ' 3 rd year  Tonbridge School Tonbridge ',
 'Kent September 2010 ',
 'July 2015 ',
 'A levels: ',
 'Chemistry  A*  ',
 'Mathematics  A  ',
 

# Process word document

In [107]:
def extract_content():
    res = {}
    for filename in word_files:
        try:
            doc = docx.Document(filename)
            content = []
            for para in doc.paragraphs:
                content.append(para.text)
            res[filename] = '\n\n'.join(content)
        except:
            with open(filename,'r', encoding="ISO-8859-1") as f:
                cv = f.read()
            content = cv.split('\n')
            b = []
            for sentence in content:
                if len(sentence)>1:
                    words = sentence.split(' ')
                    c = []
                    for word in words:
                        if len(word)<40:
                            if '\x00' not in word:
                                c.append(word)
                    if len(c) > 1:
                        b.append(' '.join(c))
            res[filename] = '\n'.join(b)
    return res

In [108]:
extracted_word = extract_content()

In [112]:
def clean_word_document():
    res = {}
    for name, content in extracted_word.items():
        replace_list = [
            ("-", " \n"), ("|", " \n"),
            ("\t", " "), (";", " "),
            (":", "")
        ]
        for pattern, replacement in replace_list:
            content = content.replace(pattern, replacement)
        x = content.split('\n')
        x = [i for i in x if len(i)>1]
        sentences = [i.rstrip() for i in x if bool(re.search('[a-zA-Z]+', i))]
        sentences = [i.strip() for i in sentences]
        sentences = remove_stopwords(sentences)
        sentences = separate_uppercase(sentences)
        sentences = clean_word(sentences)
        res[name] = sentences
    return res   

In [113]:
cleaned_word = clean_word_document()

In [114]:
cleaned_word[word_files[1]]

['Curriculum Vitae ',
 'Riccardo Biasiotti Mogliazza ',
 'Born Rome November 6th 1996 ',
 '29 Basire Street Islington ',
 'London N1 UK ',
 'Tel. 0044 7928 094082 ',
 'Via Giustino Fortunato 42 ',
 '00195 Roma Italy ',
 'Tel. 06 37350590 ',
 'Academic background ',
 '2018 Queen Mary University London Master\x92s  LLM  Banking Finance Law ',
 'Modules: Securities Regulation  74 Mergers Acquisitions  72 International Banking Law  70 EU Monetary Financial Law  70 Compliance Global Markets  68 International Economic Law  77 ',
 'Achieved Distinction every module except Compliance Global Markets  High Merit  ',
 'Dissertation: Taxation Cryptocurrencies UK USA Italy: Why important tax complex financial products  71 ',
 'LLM',
 'grade: First class honours  Distinction  ',
 'Graduation Date: 14 December 2018 ',
 '2017 Queen Mary University London School Law  London UK  ',
 'Modules: Contract law:  58 Land Law  62 Public Law  47 Criminal Law  44 EU Law  68 Administrative Law  58 Tort Law  44 Fa

In [101]:
word_files

['Cvs/444387a1_20190329140253HLS4mRMdHO.docx',
 'Cvs/ae178bd6_20190329140604CtGfnPucQP.doc',
 'Cvs/ainanshe_sabah_20190320085238.docx',
 'Cvs/Asika_Una_20190401141100.docx',
 'Cvs/Benfahr_Robert_20190402152829.doc',
 'Cvs/Dewani_Nikita_20190401102720.docx',
 'Cvs/Ebrahimi_Mahdieh_20190331120515.doc',
 'Cvs/Jeeves_Robert_20190325190640.docx',
 'Cvs/Joseph_Lini_20190320134603.docx',
 'Cvs/kawani_avan_20190331164744.docx',
 'Cvs/King_George_20190329212053.docx',
 'Cvs/Mortty_Kafui_20190402055001.docx',
 'Cvs/Piperas_Grigorios_20190326213950.docx',
 'Cvs/Solomon_Joe_20190321235309.docx',
 'Cvs/Ujoodia_Devashish_20190320134554.docx']

In [5]:
from cStringIO import StringIO
import subprocess

input_path = 'c:/test_in.pdf'

def decompress_pdf(temp_buffer):
    temp_buffer.seek(0)  # Make sure we're at the start of the file.

    process = subprocess.Popen(['pdftk.exe',
                                '-',  # Read from stdin.
                                'output',
                                '-',  # Write to stdout.
                                'uncompress'],
                                stdin=temp_buffer,
                                stdout=subprocess.PIPE,
                                stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()

    return StringIO(stdout)

ModuleNotFoundError: No module named 'cStringIO'